In [2]:
from PyPDF2 import PdfReader
import os, re, io
import tempfile
from dotenv import load_dotenv
from pdf2image import convert_from_path
import fitz
import textwrap
from collections import OrderedDict

import sys
sys.path.append('../') 
from my_rag import RAG

# Load environment variables from .env file
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

#OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
pdf_storage_dir = 'data/pdfs'
chunk_size = 500 # 500 characters default
chunk_overlap = 25 # default
top_k = 3 # number of chunks found in semantic search
DBPATH = '../data/db_file.db'
model = 'all-MiniLM-L6-v2'
question_file = '../assets/question_doc.txt'
llm_model = 'gpt-3.5-turbo'




/Users/cmmba/miniforge3/envs/nlp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def load_questions(question_file):
    qa_pairs = []
    with open(question_file, 'r') as file:
        lines = file.readlines()
    question = None
    for line in lines:
        if line.startswith('Q:'):
            question = line.strip()
        elif line.startswith('A:') and question:
            answer = line.strip()
            qa_pairs.append((question, answer))
            question = None
    print(f'A total of {len(qa_pairs)} has been successfully loaded.')
    return qa_pairs

In [4]:
def read_question_answer_pairs(file_path):
    pairs = []
    current_question = None
    current_answer = ""

    try:
        with open(file_path, 'r', encoding='ISO-8859-1') as file:
            for line in file:
                if line.startswith('Q:') and current_question:
                    pairs.append((current_question, current_answer.strip()))
                    current_question = line.strip()
                    current_answer = ""
                elif line.startswith('Q:'):
                    current_question = line.strip()
                elif line.startswith('A:'):
                    current_answer = line.strip()
                else:
                    current_answer += " " + line.strip()

            # Add the last Q-A pair if exists
            if current_question and current_answer:
                pairs.append((current_question, current_answer.strip()))

    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
            for line in file:
                if line.startswith('Q:') and current_question:
                    pairs.append((current_question, current_answer.strip()))
                    current_question = line.strip()
                    current_answer = ""
                elif line.startswith('Q:'):
                    current_question = line.strip()
                elif line.startswith('A:'):
                    current_answer = line.strip()
                else:
                    current_answer += " " + line.strip()

            # Add the last Q-A pair if exists
            if current_question and current_answer:
                pairs.append((current_question, current_answer.strip()))

    print(f'Read a total of {len(pairs)} Q-A pairs.')
    return pairs


In [5]:
def generate_rag_responses(questions, ragclass):
    responses = []
    for question in questions:
        # Generate responses
        rag_response = ragclass.generate_response(question)
        responses.append(rag_response)

    return responses

def generate_llm_responses(questions, ragclass):
    llm_responses = []
    for question in questions:
       # Generate responses
       llm_response = ragclass.integrate_llm(question)
       llm_responses.append(llm_response)
    return llm_responses


In [6]:
qa_pairs = read_question_answer_pairs(question_file)
for q, a in qa_pairs:
    print(f"Question: {q}\nAnswer: {a}\n")

Read a total of 50 Q-A pairs.
Question: Q: Where do I change my monitor configuration settings?
Answer: A: To adjust the windows that allow for viewing within the Visage 7 client, navigate to the upper left-hand corner and choose File > Preferences> Monitor Configuration. This will allow you to customize your monitors for viewing, displaying the study browser, study navigator, and export.

Question: Q: Where do I change my monitor configuration settings for viewing on a Visage 7 client?
Answer: A: To adjust the windows that allow for viewing within the Visage 7 client, navigate to the upper left-hand corner and choose File > Preferences> Monitor Configuration. This will allow you to customize your monitors for viewing, displaying the study browser, study navigator, and export.

Question: Q: How do I access priors for a patient?
Answer: A: Patient priors can be accessed from the Study Navigator, or from the Thumbnail Browser at the bottom of the viewer.

Question: Q: How do I access pri

In [7]:
# instantiate RAG class
rag = RAG(db_path = DBPATH, llm_api_key=OPENAI_API_KEY, embedding_model=model, chunk_size = chunk_size, overlap=chunk_overlap, top_k = top_k)


DatabaseError: database disk image is malformed

In [26]:
# get lists of questions and answers 
question_list = [question for (question, _) in qa_pairs]
answer_list = [answer for (_, answer) in qa_pairs]



In [8]:
# get the rag and llm responses for each question
rag_response_list = generate_rag_responses(question_list, rag)
llm_response_list = generate_llm_responses(question_list, rag)


In [10]:
# structured_output = ''
# for i, qa in enumerate(question_list):
#     structured_output += ('\n' + question_list[i])
#     structured_output += ('\n' + 'RAG response: ' + rag_response_list[i]+'\n')
#     structured_output += ('\n' + 'LLM response: ' + llm_response_list[i]+'\n') 

# with open('../assets/output_file.txt', 'w') as file:
#     file.write(structured_output)


In [25]:

# write the Q and Answers (human, rag, and llm to an output file for grading)
structured_output = ''
wrapper = textwrap.TextWrapper(width=80) 

for i, qa in enumerate(question_list):
    structured_output += ('\n' + question_list[i])
    structured_output += ('\n' + 'human answer: ' + '\n'.join(wrapper.wrap(answer_list[i])))
    structured_output += ('\n' + 'RAG response: ' + '\n'.join(wrapper.wrap(rag_response_list[i])))
    structured_output += ('\n' + 'LLM response: ' + '\n'.join(wrapper.wrap(llm_response_list[i])))
    structured_output += '\n'

with open('../assets/output_file2.txt', 'w') as file:
    file.write(structured_output)

I have manually added the scores to the QA pairs as {'rag':{'acc':1, 'rel':1, 'com': 1, 'coh':1}, 'llm':{'acc':1, 'rel':1, 'com': 1, 'coh':1}}

In [29]:
import re
import ast

# Function to extract dictionaries from a string
def extract_dictionaries(text):
    # Regular expression pattern to match dictionaries
    pattern = r'\{.*?\}\}'
    matches = re.findall(pattern, text, re.DOTALL)
    
    dicts = []
    for match in matches:
        #print(match)
        try:
             # Convert the string representation to an actual dictionary
            dict_match = ast.literal_eval(match)
            dicts.append(dict_match)
        except (SyntaxError, ValueError) as e:
            print(f"Error parsing {match}: {e}")
    
    return dicts

# Read the text file
with open('../assets/output_file2worksheet.txt', 'r') as file:
    text = file.read()

# Extract dictionaries
dicts = extract_dictionaries(text)

flattened_dicts = []
for d in dicts:
    flattened_dict = {
        'rag_Acc': d['rag']['Acc'], 
        'rag_Rel': d['rag']['Rel'],
        'rag_Com': d['rag']['Com'],
        'rag_Coh': d['rag']['Coh'],
        'llm_Acc': d['llm']['Acc'],
        'llm_Rel': d['llm']['Rel'],
        'llm_Com': d['llm']['Com'],
        'llm_Coh': d['llm']['Coh']
    }
    flattened_dicts.append(flattened_dict)

# Convert to DataFrame
df = pd.DataFrame(flattened_dicts)
df['Question'] = question_list
# Display the DataFrame
df.head()



,rag_Acc,rag_Rel,rag_Com,rag_Coh,llm_Acc,llm_Rel,llm_Com,llm_Coh,Question
0,3,3,3,3,1,1,2,3,Q: Where do I change my monitor configuration ...
1,3,3,3,3,2,3,2,3,Q: Where do I change my monitor configuration ...
2,3,3,2,3,1,1,3,3,Q: How do I access priors for a patient?
3,3,3,3,3,3,3,2,3,Q: How do I access priors for a patient using ...
4,3,3,3,3,1,1,2,3,Q: How can I customize my toolbar?


In [36]:
df.describe()


,rag_Acc,rag_Rel,rag_Com,rag_Coh,llm_Acc,llm_Rel,llm_Com,llm_Coh
count,50.00000,50.000000,50.000000,50.0,50.000000,50.000000,50.00000,50.0
mean,2.52000,2.800000,2.960000,3.0,1.660000,2.100000,2.86000,3.0
std,0.81416,0.571429,0.197949,0.0,0.823383,0.909137,0.35051,0.0
min,1.00000,1.000000,2.000000,3.0,1.000000,1.000000,2.00000,3.0
25%,2.00000,3.000000,3.000000,3.0,1.000000,1.000000,3.00000,3.0
50%,3.00000,3.000000,3.000000,3.0,1.000000,2.000000,3.00000,3.0
75%,3.00000,3.000000,3.000000,3.0,2.000000,3.000000,3.00000,3.0
max,3.00000,3.000000,3.000000,3.0,3.000000,3.000000,3.00000,3.0


In [39]:
# find rows where question mentions visage7
rows_with_visage7 = df[df['Question'].str.contains('Visage7', na=False)]

# Display the filtered rows
print(rows_with_visage7)

    rag_Acc  rag_Rel  rag_Com  rag_Coh  llm_Acc  llm_Rel  llm_Com  llm_Coh  \
3         3        3        3        3        3        3        2        3   
5         3        3        3        3        3        3        3        3   
7         3        3        3        3        3        3        3        3   
11        3        3        3        3        3        3        3        3   
13        1        1        3        3        1        1        3        3   
19        3        3        3        3        3        3        3        3   
21        3        3        3        3        2        3        3        3   
23        3        3        3        3        3        3        3        3   
26        3        3        3        3        2        3        3        3   
33        3        3        3        3        3        3        3        3   
35        3        3        3        3        3        3        3        3   
49        3        3        3        3        3        3        

In [45]:
## do statistics for qa pairs that are with and without the software name for context
relevant_rows = [3, 5, 7, 11, 13, 19, 21, 23, 26, 33, 35, 49]
without_rows = [2, 4, 6, 10, 12, 18, 20, 22, 25, 32, 34, 48]
df_with_context = df.loc[relevant_rows]
df_without_context = df.loc[without_rows]


In [44]:
# for questions where not provided software name
df_without_context.describe()

,rag_Acc,rag_Rel,rag_Com,rag_Coh,llm_Acc,llm_Rel,llm_Com,llm_Coh
count,12.000000,12.000000,12.000000,12.0,12.000000,12.000000,12.000000,12.0
mean,2.416667,2.666667,2.916667,3.0,1.416667,1.666667,2.833333,3.0
std,0.900337,0.778499,0.288675,0.0,0.668558,0.887625,0.389249,0.0
min,1.000000,1.000000,2.000000,3.0,1.000000,1.000000,2.000000,3.0
25%,1.750000,3.000000,3.000000,3.0,1.000000,1.000000,3.000000,3.0
50%,3.000000,3.000000,3.000000,3.0,1.000000,1.000000,3.000000,3.0
75%,3.000000,3.000000,3.000000,3.0,2.000000,2.250000,3.000000,3.0
max,3.000000,3.000000,3.000000,3.0,3.000000,3.000000,3.000000,3.0


In [46]:
# for questions where provided software name
df_with_context.describe()

,rag_Acc,rag_Rel,rag_Com,rag_Coh,llm_Acc,llm_Rel,llm_Com,llm_Coh
count,12.000000,12.000000,12.0,12.0,12.000000,12.000000,12.000000,12.0
mean,2.833333,2.833333,3.0,3.0,2.666667,2.833333,2.916667,3.0
std,0.577350,0.577350,0.0,0.0,0.651339,0.577350,0.288675,0.0
min,1.000000,1.000000,3.0,3.0,1.000000,1.000000,2.000000,3.0
25%,3.000000,3.000000,3.0,3.0,2.750000,3.000000,3.000000,3.0
50%,3.000000,3.000000,3.0,3.0,3.000000,3.000000,3.000000,3.0
75%,3.000000,3.000000,3.0,3.0,3.000000,3.000000,3.000000,3.0
max,3.000000,3.000000,3.0,3.0,3.000000,3.000000,3.000000,3.0
